```{contents}
```

## Few-shot Prompting 



In **LangChain**, **few-shot prompting** is a technique where you **provide example input–output pairs** inside the prompt so the model can **learn the pattern at inference time**.

> The model is not trained.
> It is **conditioned by examples embedded in the prompt**.

LangChain treats few-shot prompts as **structured prompt objects**, not raw text.

---

### Why Few-shot Prompting Is Important

Few-shot prompting helps when:

* Zero-shot responses are inconsistent
* Output format must be followed strictly
* Classification labels are limited
* Tool usage needs guidance
* Fine-tuning is not available or overkill

---

### Where Few-shot Prompting Fits in LangChain

```
Examples
   ↓
PromptTemplate / ChatPromptTemplate
   ↓
ChatModel
   ↓
Consistent Output
```

Few-shot prompting is implemented **at the prompt layer**, not the model layer.

---

### Few-shot Prompting vs Fine-tuning

| Aspect      | Few-shot Prompting     | Fine-tuning   |
| ----------- | ---------------------- | ------------- |
| Training    | No                     | Yes           |
| Latency     | Higher (longer prompt) | Lower         |
| Flexibility | High                   | Low           |
| Cost        | Token-based            | Training cost |
| Deployment  | Immediate              | Slow          |

---

### Few-shot Prompting with PromptTemplate (Text Models)

#### Defining Examples



In [1]:

from langchain_core.prompts import PromptTemplate

examples = """
Input: Email service is down
Category: Infrastructure

Input: User cannot reset password
Category: Access
"""

prompt = PromptTemplate.from_template(
    """
Classify the issue.

{examples}

Input: {input}
Category:
"""
)


### Invocation


In [3]:
from langchain_openai import ChatOpenAI
chain = prompt | ChatOpenAI()
chain.invoke({
    "examples": examples,
    "input": "VPN is not working"
})

AIMessage(content='Connectivity', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 44, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cp881svQa6jgzVVAQPPdHERTPhisy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b3fcd-c94d-7ac2-b69b-fb34e0dd4eb6-0', usage_metadata={'input_tokens': 44, 'output_tokens': 2, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


---

### Few-shot Prompting with ChatPromptTemplate (Recommended)

#### Static Few-shot Examples



In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an IT ticket classifier."),
    ("human", "Email service is down"),
    ("assistant", "Category: Infrastructure"),
    ("human", "User cannot reset password"),
    ("assistant", "Category: Access"),
    ("human", "{input}")
])




### Invocation



In [7]:
chain = prompt | ChatOpenAI()
chain.invoke({"input": "VPN is slow"})


AIMessage(content='Category: Connectivity', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 51, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cp88wORoy6IHI8deTJrcGt2APx1A0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b3fce-a962-7cd0-ab98-8ff47e46754b-0', usage_metadata={'input_tokens': 51, 'output_tokens': 3, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


---

### Dynamic Few-shot Prompting (FewShotPromptTemplate)

#### When Examples Are Data-driven

LangChain provides `FewShotPromptTemplate`.



In [8]:

from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nCategory: {output}"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=[
        {"input": "Email down", "output": "Infrastructure"},
        {"input": "Password reset issue", "output": "Access"},
    ],
    example_prompt=example_prompt,
    prefix="Classify the issue.",
    suffix="Input: {input}\nCategory:",
    input_variables=["input"]
)




### Invocation



In [10]:
chain = few_shot_prompt | ChatOpenAI()
chain.invoke({"input": "VPN is not working"})


AIMessage(content='Infrastructure', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 40, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cp89iCvwhifPBmZNdAenp2OfLMbly', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b3fcf-6201-7962-ab19-a9c06f739579-0', usage_metadata={'input_tokens': 40, 'output_tokens': 1, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})



---

### Few-shot Prompting with Chat Models (Dynamic)

#### Using FewShotChatMessagePromptTemplate



In [11]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "Email down", "output": "Infrastructure"},
    {"input": "Password reset failed", "output": "Access"},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("assistant", "{output}")
])

few_shot_chat_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an IT ticket classifier."),
    few_shot_chat_prompt,
    ("human", "{input}")
])




---

### Example Selection (Advanced)

#### Semantic Example Selection

```python
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
```

LangChain can:

* Embed examples
* Select the most relevant few
* Reduce prompt size
* Improve accuracy

---

### Few-shot Prompting in RAG

Few-shot examples:

* Guide answer style
* Enforce citations
* Control hallucinations

```python
("system", "Answer using the context. Follow the examples.")
```

---

### Common Mistakes

#### Too many examples

❌ Increases latency and cost

#### Poor-quality examples

❌ Reinforces wrong behavior

#### Mixing formats

❌ Confuses the model

#### Using few-shot when zero-shot is enough

❌ Wastes tokens

---

#### Best Practices

* 2–5 high-quality examples
* Keep formats identical
* Use domain-relevant examples
* Combine with structured output when possible
* Prefer ChatPromptTemplate for chat models

---

### When to Use Few-shot Prompting

| Use Case              | Recommended |
| --------------------- | ----------- |
| Classification        | Yes         |
| Structured extraction | Yes         |
| Tool usage guidance   | Yes         |
| Open-ended chat       | Usually No  |
| Simple Q&A            | No          |

---

### Interview-Ready Summary

> “Few-shot prompting in LangChain is implemented at the prompt layer using PromptTemplate or ChatPromptTemplate, where example input–output pairs are embedded to guide model behavior without fine-tuning.”

---

### Rule of Thumb

* **Unstable outputs → Few-shot**
* **Stable task → Zero-shot**
* **Complex task → Few-shot + structured output**
* **Long-term behavior → Fine-tuning**
